# Droptout
## Overfitting
- 과적합 → 너무 학습된 데이터 위로 학습한 경우
- 트레인 셋의 loss가 적겠지만 test_set은 loss가 많이 날 경우가 있음
- Underfitting → 학습이 덜 될 경우

## Solutions for overfitting
- More training data
- Reduce the number of features
- Regularization
- Dropout !

## Dropout
 - 주의점  train_set은 드랍아웃 사용해야하지만, test_set은 사용 못하게 해야함


In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

In [4]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# nn Layers
linear1 = torch.nn.Linear(784, 512, bias = True)
linear2 = torch.nn.Linear(512, 512, bias = True)
linear3 = torch.nn.Linear(512, 512, bias = True)
linear4 = torch.nn.Linear(512, 512, bias = True)
linear5 = torch.nn.Linear(512, 10, bias = True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p= drop_prob)

#model
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

In [7]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0210, -0.0870,  0.0975,  ..., -0.0388,  0.0134,  0.0955],
        [-0.0414, -0.0873, -0.0093,  ...,  0.0386, -0.0246,  0.0559],
        [-0.0025,  0.0337, -0.0765,  ...,  0.0589, -0.0037, -0.0802],
        ...,
        [-0.0863,  0.0736,  0.0186,  ...,  0.0871, -0.0761,  0.0289],
        [ 0.0616,  0.1032, -0.0644,  ...,  0.0756, -0.0918, -0.0351],
        [ 0.0032, -0.0491,  0.0938,  ...,  0.0228,  0.0561, -0.0352]],
       device='cuda:0', requires_grad=True)

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device) # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
total_batch = len(data_loader)
model.train() # set the model to train mode (dropout= True)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning finished')

Epoch: 0001 cost = 0.313976884
Epoch: 0002 cost = 0.149540097
Epoch: 0003 cost = 0.110147350
Epoch: 0004 cost = 0.095629260
Epoch: 0005 cost = 0.079806544
Epoch: 0006 cost = 0.074046530
Epoch: 0007 cost = 0.070271045
Epoch: 0008 cost = 0.061384164
Epoch: 0009 cost = 0.058667608
Epoch: 0010 cost = 0.055732798
Epoch: 0011 cost = 0.051962510
Epoch: 0012 cost = 0.051793754
Epoch: 0013 cost = 0.046914455
Epoch: 0014 cost = 0.045555864
Epoch: 0015 cost = 0.043304753
Learning finished


In [10]:
# Test model and check accurancy
with torch.no_grad():
    model.eval() # set the model to evaluation mode (dropout = False)
    
    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r+1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.981499969959259
Label:  8
Prediction:  8


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
